## Deploy Azure ML Models to IoT Edge Devices - Using Python SDK
You can use the Azure Machine Learning Python SDK to deploy models to IoT Edge devices if your IoT Hub is in the same subscription as your Azure ML Workspace. Otherwise, you will need to deploy using the Azure IoT CLI or the Azure Portal. 

### Prerequisites
* An IoT Edge Device (specifically, the Data Box Edge machine with Compute VM enabled)
    * Follow [steps 1 - 5a](https://docs.microsoft.com/en-us/azure/databox-online/data-box-edge-deploy-prep) to set up your Data Box Edge machine and enable the Linux Compute VM and an associated IoT Hub. This IoT Hub will deploy the AccelContainerImage to the Compute VM. **Note:** In order to use the Azure ML SDK to deploy your model to your IoT Hub, you must create your Data Box Edge resource in the same subscription as your Azure ML Workspace.
* An Azure ML model registered OR a ContainerImage successfully created in your Workspace.
    * For an Accelerated model, you can follow [this notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/accelerated-models/accelerated-models-quickstart.ipynb) through step 3. Step 4 and Step 5 are optional, and will be completed behind the scenes in deployment if not done.

### Steps
1. Retrieve your Azure ML Workspace 
* Retrieve your IoT Hub's connection string
* Attach your IoT Hub as Azure ML Compute
* Configure Azure ML module
* Configure Sample Client module
* Deploy your Model or Image
* Stop Sample Client

### 1. Retrieve your Azure ML Workspace

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

### 2. Retrieve your IoT Hub's connection string

You do not need to follow this if you do not want to install the Azure CLI and are willing to access the IoT Hub connection string via the Azure Portal.

You can install the necessary CLI packages in this notebook by running the below cell, if not already installed.

In [ ]:
%%bash
pip install azure-cli # Install Azure CLI
az extension add --name azure-cli-iot-ext # Add IoT CLI extension
pip install -U jupyter_console # Fix Jupyter dependencies overridden by Azure CLI installation
pip install docker # Install Docker
pip install azureml-accel-models[cpu]
az login # Log into Azure account

Set your Data Box Edge's resource group, IoT Hub name, and IoT Edge device name. If you're using the Azure ML SDK, we assume as a prerequisite that the subscription of your IoT Hub is the same as the Azure ML Workspace.

In [ ]:
# Set the resource group where your IoT Hub resides. This can be different from your Azure ML Workspace.
resource_group = ws.resource_group

# Set the subscription where your IoT Hub resides. This should be the same as your Azure ML Workspace
subscription_id = ws.subscription_id

# Your Data Box Edge IoT Hub name
iot_hub_name = "dummy-dbe"
# Your Data Box Edge IoT Edge Device
iot_device_id = "computevm"

Set subscription and resource group defaults, then get connection string for IoT Hub.

In [ ]:
!az account set --subscription $subscription_id
!az configure --defaults group=$resource_group
ret = !az iot hub show-connection-string -n $iot_hub_name -o tsv
connection_string = ret[0]

### 3. Attach your IoT Hub as Azure ML Compute

If your IoT Hub is not attached to the Azure ML Workspace, you can attach it by running the below command.

In [ ]:
from azureml.contrib.core.compute import IotHubCompute

# If you haven't attached your Data Box Edge's IoT Hub as compute
config = IotHubCompute.attach_configuration(name=iot_hub_name, resource_group=resource_group, connection_string=connection_string)
iothub_compute = IotHubCompute.attach(ws, iot_device_id, config)
iothub_compute.wait_for_completion()

Or if you've already attached your IoT Hub as compute, run the below command to pull the existing Azure ML IotHubCompute

In [ ]:
# from azureml.contrib.core.compute import IotHubCompute

# iothub_compute = IotHubCompute(ws, iot_device_id)

List Azure ML Compute Targets.

In [ ]:
from azureml.core.compute import ComputeTarget

compute_targets = ComputeTarget.list(ws)
for t in compute_targets: 
    if t.type == "IotHub":
        print("IotHub '{}' has provisioning state '{}'.".format(t.name, t.provisioning_state))

### 4. Configure Azure ML module

The below code provides information for the AccelContainerImage to run on the FPGA-enabled Data Box Edge. 

The ``container_config`` for accelerated models allows access to the FPGA device and exposes the port 50051. You can use the default ``container_config`` provided in the code.

For CPU models, use the following ``container_config``, adjusting the port as you like:

```
container_config = '{ \
  "HostConfig": { \
    "PortBindings": { \
      "80/tcp": [ \
        { \
          "HostPort": "80" \
        } \
      ] \
    } \
  } \
}'
```

The ``routes`` are the default IoT Edge deployment routes which route any messages sent from modules to the IoT Hub.

In [ ]:
from azureml.contrib.core.webservice import IotWebservice, IotBaseModuleSettings, IotModuleSettings

#Pick a module name
module_name = "resnet50-host-2"

container_config = '{ \
  "HostConfig": { \
    "Binds": [ \
      "/etc/hosts:/etc/hosts" \
    ], \
    "Privileged": true, \
    "Devices": [ \
      { \
        "PathOnHost": "/dev/catapult0", \
        "PathInContainer": "/dev/catapult0" \
      }, \
      { \
        "PathOnHost": "/dev/catapult1", \
        "PathInContainer": "/dev/catapult1" \
      } \
    ], \
    "PortBindings": { \
      "50051/tcp": [ \
        { \
          "HostPort": "50051" \
        } \
      ] \
    } \
  } \
}'

routes = {
    "route": "FROM /messages/* INTO "
}

# Here, we define the Azure ML module with the container_config options above
aml_module = IotBaseModuleSettings(name=module_name, create_option=container_config)

### 6. Configure Sample Client module

You can also deploy modules that are not created through Azure ML, for example to deploy a client.

#### Build sample client image and push to your ACR
The Docker image built from the /sample-client directory will work with the ResNet50 image created in Quickstart. To create a sample client that works with the Image created from transfer learning, update the line to below. The /sample-client uses ImageNet pictures for inferencing; the /sample-client-tl handles the response differently and uses images of cats and dogs. 

```
client_app = "sample-client-tl"
```

In [ ]:
from azureml.core import ContainerRegistry
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azure.mgmt import containerregistry

acr = ws.get_details().get("containerRegistry")
address = acr.split("/")[-1]

# The default sample client works with the Docker image created from Quickstart
client_app = "sample-client"
client_version = 1
client_url = "{}.azurecr.io/{}:{}".format(address, client_app, client_version)

print(address, client_app, client_url, sep = '\n')

# Here, we use Azure CLI to sign into the ACR. Otherwise, you can sign in manually using docker login and your ACR credentials
# !docker login $address -u $username -p $password
!az acr login --name $address --subscription $subscription_id --resource-group $resource_group
!docker build -t $client_url -f ./Dockerfile ./$client_app
!docker push $client_url

In [ ]:
client_env = { 
    "DEVICE_CONNECTION_STRING": connection_string
}

client_container_config = '{  \
    "Tty": true, \
    "Cmd": [ \
        "--input-tensors", \
        "Placeholder:0", \
        "--output-tensors", \
        "classifier/resnet_v1_50/predictions/Softmax:0", \
        "--wait", \
        "10", \
    ] \
}'

aml_client = IotModuleSettings(name='azureml-client',
                               image_location='{}:1.0'.format(client_url),
                               env=client_env,
                               create_option=client_container_config)

### 7. Deploy your Model or Image

There are two scenarios for deployment, depending on whether or not your model is packaged into the AccelContainerImage. 

If you haven't created an AccelContainerImage yet, you can deploy from the model by specifying the inference_config. This will create an AccelContainerImage (or ContainerImage for CPU) behind the scenes each time. That AccelContainerImage is stored in your Azure Container Registery (ACR) associated with the Azure ML Workspace.

#### a. Deploy from a Model that needs to be converted
For deploying Accelerated Models, we will create an [AccelInferenceConfig](https://docs.microsoft.com/en-us/python/api/azureml-accel-models/azureml.accel.accelinferenceconfig?view=azure-ml-py) that will be used to create the AccelContainerImage behind the scenes. In the case of a model that is meant for CPU only, refer to [documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py) to create an InferenceConfig to create a regular ContainerImage.

For AccelInferenceConfig, if your model is not yet converted to the Accelerated Onnx format, you can provide the ``input_tensor`` and ``output_tensor``. In the case that your model has already been converted, then you can create an empty ``AccelInferenceConfig()`` object.

Then define the deployment manifest and the model you want to deploy.

In [ ]:
from azureml.accel import AccelInferenceConfig
from azureml.core.model import Model

# The input and output tensors are available in the Azure ML Hardware Accelerated notebooks. 
# These are the defaults from the Resnet50 Quickstart notebook.
inference_config = AccelInferenceConfig(input_tensor="Placeholder:0", output_tensor="classifier/resnet_v1_50/predictions/Softmax:0")

# Create deployment manifest
deploy_config = IotWebservice.deploy_configuration(device_id=iot_device_id, routes=routes, aml_module=aml_module, external_modules=[aml_client])

# Get the model you want to deploy. This can be the CPU version of the model, 
# an Accelerated Model before conversion, or an Accelerated Model after conversion.
model_name = "resnet50"
registered_model = Model(ws, name=model_name)

# Deploy from model, using module_name as your IotWebservice name
iot_service_name = module_name

iot_service = Model.deploy(ws, iot_service_name, [registered_model], inference_config, deploy_config, iothub_compute)
iot_service.wait_for_deployment()

#### b. Deploy from ContainerImage

If you've already created an AccelContainerImage with the converted model that you want to run, then you can use the cell below to pull that image and deploy it. This image is stored in your Azure Container Registry (ACR) associated with your Azure ML Workspace.

In [ ]:
from azureml.core import Image, Webservice
from azureml.contrib.core.webservice import IotWebservice
from azureml.accel import AccelContainerImage

# Then we define the deployment manifest for our IoT Edge device with the aml_module and routes
deploy_config = IotWebservice.deploy_configuration(device_id=iot_device_id, routes=routes, aml_module=aml_module, external_modules=[aml_client])

# Deploy from latest version of image, using module_name as your IotWebservice name
image_name = "resnet50-image"
iot_service_name = module_name

# Can specify version=x, otherwise will grab latest
image = Image(ws, image_name) 
iot_service = IotWebservice.deploy_from_image(ws, iot_service_name, image, deploy_config, iothub_compute)
iot_service.wait_for_deployment()

### 8. Stop Sample Client
First, delete the previous iot_service, then re-deploy without the sample client so that it doesn't use power for inferencing.

In [ ]:
iot_service.delete()

When creating the IotWebservice deployment configuration, we will leave off the external module from the deployment manifest.

In [ ]:
from azureml.core import Image, Webservice
from azureml.contrib.core.webservice import IotWebservice
from azureml.accel import AccelContainerImage

# This time, we will leave off the external module from the deployment manifest
deploy_config = IotWebservice.deploy_configuration(device_id=iot_device_id, routes=routes, aml_module=aml_module)

# Deploy from latest version of image, using module_name as your IotWebservice name
image_name = "resnet50-image"
iot_service_name = module_name

# Can specify version=x, otherwise will grab latest
image = Image(ws, image_name) 
iot_service = IotWebservice.deploy_from_image(ws, iot_service_name, image, deploy_config, iothub_compute)
iot_service.wait_for_deployment()